In [1]:
import pandas as pd
import requests as r
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time
import numpy as np
import json
import unicodedata
import pymysql
from sqlalchemy import create_engine

In [3]:
## Get the dataframe to complete
cities=pd.read_csv('../data/medical_desert_clean.csv') 
cities.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CODGEO,Communes,APL,P16_POP,median_living_standard,healthcare_education_establishments,density_area,annual_pop_growth,unemployment_rate,secondary_residence_rate,...,mobility_rate,average_birth_rate,CSP1_rate,CSP2_rate,CSP3_rate,CSP4_rate,CSP5_rate,CSP6_rate,CSP7_rate,CSP8_rate
0,01001,L'Abergement-Clémenciat,2.396,767,22679.000000,0,48.087774,-0.335578,7.127430,4.597701,...,2.216428,1.060116,2.479339,3.305785,12.396694,15.702479,16.528926,20.661157,23.966942,4.958678
1,01002,L'Abergement-de-Varey,2.721,243,24382.083333,0,26.557377,0.757662,6.944444,30.769231,...,2.057613,1.761006,0.000000,10.256410,7.692308,12.820513,20.512821,5.128205,33.333333,10.256410
2,01004,Ambérieu-en-Bugey,4.335,14081,19721.000000,0,572.398374,0.347315,12.038385,1.684887,...,1.516341,1.595989,0.024879,2.662394,6.939410,17.209926,16.240671,15.940930,24.740051,16.241738
3,01005,Ambérieux-en-Dombes,4.279,1671,23378.000000,0,104.962312,0.872154,6.348660,1.810755,...,0.985957,1.235096,0.378011,4.511481,7.896554,17.271010,18.019503,17.254154,23.429304,11.239984
4,01006,Ambléon,0.912,110,21660.000000,0,18.707483,-0.359722,11.111111,16.216216,...,2.727273,1.621622,0.000000,0.000000,5.555556,27.777778,16.666667,16.666667,27.777778,5.555556


In [62]:
# Getting the match between INSEE geocode and postal code
url="https://data.opendatasoft.com/api/v2/opendatasoft/datasets/correspondance-code-insee-code-postal%40public/exports/json?rows=-1&pretty=false&timezone=UTC"
    
result=r.get(url).json() 
match_code=pd.json_normalize(result)
print("Shape:",match_code.shape)
match_code.head()


Shape: (36742, 20)


,code_comm,nom_dept,statut,z_moyen,nom_region,code_reg,insee_com,code_dept,postal_code,id_geofla,superficie,code_cant,nom_comm,code_arr,population,geo_point_2d.lat,geo_point_2d.lon,geo_shape.geometry.type,geo_shape.geometry.coordinates,geo_shape.type
0,318,[MEURTHE-ET-MOSELLE],[Commune simple],263.0,[LORRAINE],41,54318,54,54460,3065,2530.0,12,LIVERDUN,4,6.0,48.753717,6.052354,Polygon,"[[[6.075779828649081, 48.7208969140098], [6.07...",Feature
1,293,[ARDECHE],[Commune simple],518.0,[RHONE-ALPES],82,07293,07,07130,8760,1425.0,19,SAINT-ROMAIN-DE-LERPS,3,0.7,44.976308,4.796264,Polygon,"[[[4.79284159231402, 44.958619727159856], [4.7...",Feature
2,086,[NORD],[Commune simple],64.0,[NORD-PAS-DE-CALAIS],31,59086,59,59299,29952,1375.0,60,BOESCHEPE,4,2.2,50.798998,2.699370,Polygon,"[[[2.738555843482581, 50.780591488369744], [2....",Feature
3,221,[MAYENNE],[Commune simple],210.0,[PAYS DE LA LOIRE],52,53221,53,53600,528,2047.0,11,SAINT-GEORGES-SUR-ERVE,2,0.4,48.177273,-0.292545,Polygon,"[[[-0.24663584080271903, 48.16015118223144], [...",Feature
4,154,[EURE],[Commune simple],147.0,[HAUTE-NORMANDIE],23,27154,27,27220,33185,1848.0,32,CHAVIGNY-BAILLEUL,3,0.5,48.880066,1.196227,Polygon,"[[[1.18962917841442, 48.85857938857242], [1.16...",Feature


In [70]:
match_code[match_code.postal_code=='42240'][['postal_code','insee_com','nom_comm']]

,postal_code,insee_com,nom_comm
6484,42240,42316,UNIEUX
21959,42240,42270,SAINT-PAUL-EN-CORNILLON
28558,42240,42262,SAINT-MAURICE-EN-GOURGOIS
34614,42240,42031,CALOIRE


In [27]:
# Connect to doctolib database
username='root'
host='localhost'
database='doctolib_scrap'
password=''

engine=create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

In [28]:
query="SELECT zipcode, count(distinct link) FROM doctors WHERE speciality = 'Médecin généraliste' GROUP BY 1"
db=pd.read_sql_query(query,engine)

db

,zipcode,count(distinct link)
0,,1
1,55270,1
2,6 Avenue Du 8 Mai 1945,1
3,?????????????????,1
4,01000,104
...,...,...
5554,OISY LE VERGER,1
5555,SAINS EN GOHELLE,1
5556,Saint Julien en Born,1
5557,VENDIN LE VIEIL,2


In [31]:
db['count(distinct link)'].sum()

60313

In [60]:
db[db.zipcode=='42240']

,zipcode,count(distinct link)
2222,42240,4


In [29]:
# Reading the dataset
eqmt = pd.read_csv('../data/raw_data/bpe18_ensemble_csv/bpe18_ensemble.csv',sep=';')
print("Shape:",eqmt.shape)
eqmt.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape: (1035564, 7)


,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,NB_EQUIP
0,84,1,1001,01001,2018,A401,2
1,84,1,1001,01001,2018,A404,4
2,84,1,1001,01001,2018,A504,1
3,84,1,1001,01001,2018,A507,1
4,84,1,1001,01001,2018,B203,1


In [51]:
education_health = eqmt[eqmt.TYPEQU=='D201'].groupby('DEPCOM').NB_EQUIP.agg('sum').reset_index()
education_health

,DEPCOM,NB_EQUIP
0,1004,27
1,1005,2
2,1007,2
3,1014,2
4,1021,3
...,...,...
9554,31065,1
9555,31066,8
9556,31069,31
9557,31073,3


In [66]:
# Renaming DEPCOM as CODGEO to match with merge dataframe
education_health = education_health.rename(columns={'DEPCOM':'CODGEO'})

# Cleaning codes to add 0 when code is only 4 number
education_health.CODGEO = education_health.CODGEO.apply(lambda x: '0'+str(x) if len(str(x))==4 else x).astype(str).copy()

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
education_health = education_health.convert_dtypes()

In [72]:
education_health[education_health.CODGEO=='42262']

,CODGEO,NB_EQUIP
2539,42262,1
